In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from datetime import datetime, timedelta
import time

# Dictionary to store input timestamps and count
input_records = {
    'last_input_time': None,
    'input_count': 0
}

def get_manual_input():
    global input_records
    current_time = datetime.now()
    
    # Reset input count after 24 hours
    if input_records['last_input_time'] and (current_time - input_records['last_input_time']) >= timedelta(hours=24):
        input_records['input_count'] = 0
    
    # Restrict input to max 3 times per 24 hours
    if input_records['input_count'] >= 3:
        print("You have reached the maximum input limit for today. Please try again tomorrow.")
        time.sleep(60 * 60)  # Sleep for 1 hour to prevent immediate retries
        return None
    
    input_records['last_input_time'] = current_time  # Update last input time
    input_records['input_count'] += 1  # Increment input count
    
    try:
        date_str = input("Enter the date (YYYY-MM-DD): ")
        date = datetime.strptime(date_str, '%Y-%m-%d')
        hba1c = float(input("Enter HbA1c value: "))
        blood_pressure = input("Enter Blood Pressure (e.g., 120/80): ")
        cholesterol = int(input("Enter Cholesterol level: "))
        bmi = float(input("Enter BMI: "))
        
        return {
            'Date': date,
            'HbA1c': hba1c,
            'BloodPressure': blood_pressure,
            'Cholesterol': cholesterol,
            'BMI': bmi
        }
    except ValueError:
        print("Invalid input format. Please enter correct values.")
        return None

def analyze_patient_data(lab_data, model):
    features = [lab_data['HbA1c'], int(lab_data['BloodPressure'].split('/')[0]), lab_data['Cholesterol'], lab_data['BMI']]
    prediction = model.predict([features])[0]
    recommendations = []
    observations = []
    
    if prediction == 'Diabetes':
        recommendations.append("Maintain a healthy diet, exercise regularly, and monitor blood sugar levels.")
        if lab_data['HbA1c'] >= 6.5:
            observations.append("High HbA1c indicates confirmed diabetes.")
            recommendations.append("May consider a retinoscopy and KFT.")
        elif lab_data['HbA1c'] >= 5.7:
            observations.append("Pre-diabetic stage")
        else:
            observations.append("Not a high range, still be careful")
        observations.append("Take prescribed diabetes medication.")
    
    if prediction == 'Hypertension':
        recommendations.append("Reduce salt intake, exercise regularly, and monitor blood pressure levels.")
        if int(lab_data['BloodPressure'].split('/')[0]) >= 140:
            observations.append("High blood pressure indicates advanced hypertension.")
        else:
            observations.append("Moderate blood pressure levels indicate controlled hypertension.")
        observations.append("Take prescribed hypertension medication.")
    
    if prediction == 'Both':
        recommendations.append("Follow a balanced diet, exercise regularly, and monitor both blood pressure and blood sugar levels.")
        if lab_data['HbA1c'] >= 6.5 and int(lab_data['BloodPressure'].split('/')[0]) >= 140:
            observations.append("Signs of both confirmed diabetes and hypertension detected.")
        else:
            observations.append("Signs of both moderate diabetes and hypertension detected.")
        observations.append("Take prescribed hypertension medication.")
    
    if prediction == 'None':
        recommendations.append("Maintain a healthy lifestyle to prevent chronic diseases.")
        observations.append("No chronic diseases detected. You are Healthy.")
    
    return prediction, recommendations, observations

# Load dataset
df = pd.read_csv('health_dataset_1.csv')

# Adjust column names based on actual names in the dataset
X = df[['HbA1c', 'Blood Pressure (mm Hg)', 'Cholesterol (mg/dL)', 'BMI']]
X['Blood Pressure (mm Hg)'] = X['Blood Pressure (mm Hg)'].apply(lambda x: int(x.split('/')[0]))
y = df['Disease']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

data = get_manual_input()
if data:
    disease, recommendations, observations = analyze_patient_data(data, model)
    print("Disease:", disease)
    print("Recommendations:", recommendations)
    print("Observations:")
    for observation in observations:
        print("-", observation)
    print("Consult your doctor for further analysis and guidance.")

C:\Users\shiva\AppData\Local\Temp\ipykernel_12676\552730837.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Blood Pressure (mm Hg)'] = X['Blood Pressure (mm Hg)'].apply(lambda x: int(x.split('/')[0]))


Enter the date (YYYY-MM-DD):  2025-02-08
Enter HbA1c value:  4.5
Enter Blood Pressure (e.g., 120/80):  150/90
Enter Cholesterol level:  210
Enter BMI:  25.5


Disease: Hypertension
Recommendations: ['Reduce salt intake, exercise regularly, and monitor blood pressure levels.']
Observations:
- High blood pressure indicates advanced hypertension.
- Take prescribed hypertension medication.
Consult your doctor for further analysis and guidance.


C:\Users\shiva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
